In [7]:
import os
import argparse
import random
import numpy as np 

import torch
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from torch.autograd import Variable

# Processed dataset is by default in the "dataset/" folder

import torchvision
from torchvision import transforms

def get_splits(dataset, train_split=0.8, test_split=0.15):

    '''
    Obtain train/test/validation splits from torch dataset.
    '''

    n = len(dataset)
    train_size, test_size = int(train_split * n), int(test_split * n)
    val_size = n - train_size - test_size

    train_dataset, test_dataset, val_dataset = data.random_split(dataset, [train_size, test_size, val_size])

    return train_dataset, test_dataset, val_dataset


In [8]:
# Imitaded args just for this notebook

parser = argparse.ArgumentParser(description="Snakes")

parser.add_argument('--path', type=str, default="dataset", help="Dataset folder path")
parser.add_argument('--batch-size', type=int, default=10, help='Input batch size for training')
parser.add_argument('--resize', type=bool, default=True, help='Resize width')
parser.add_argument('--width', type=int, default=224, help='Resize width')
parser.add_argument('--height', type=int, default=224, help='Resize height')

# Val split is the remaining part
parser.add_argument('--train_split', type=float, default=0.8, help='Train split ratio')
parser.add_argument('--test_split', type=float, default=0.15, help='Test split ratio')

parser.add_argument('--model', type=str, help="Model name")
parser.add_argument('--weight-decay', type=float, default=0.0, help='Weight decay hyperparameter')
parser.add_argument('--lr', type=float, default = 1e-3, metavar='LR', help='learning rate')
parser.add_argument('--lr_step_size', type=int, default = 50, help='learning rate scheduler step size')
parser.add_argument('--lr_step_gamma', type=int, default = 0.1, help='learning rate step gamma')
parser.add_argument('--epochs', type=int, default=10, help='number of epochs to train')

parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='number of batches between logging train status')

# Need to pass a string to make it work in Jupyter
args = parser.parse_args("")

args.cuda = torch.cuda.is_available()

#device = torch.device("cuda" if args.cuda else "cpu")
#kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

random_seed = 128

if args.cuda:
    torch.cuda.empty_cache() 
    kwargs = {'num_workers': 1, 'pin_memory': True}
    device = "cuda"
    torch.cuda.manual_seed(random_seed)
else:
    kwargs = {}
    device = "cpu"

# TODO: Do I need to call it twice here?
np.random.seed(random_seed)

print(device)
print(args)

cuda
Namespace(batch_size=10, cuda=True, epochs=10, height=224, log_interval=10, lr=0.001, lr_step_gamma=0.1, lr_step_size=50, model=None, path='dataset', resize=True, test_split=0.15, train_split=0.8, weight_decay=0.0, width=224)


In [9]:
n_classes = len(os.listdir(args.path))

snakes_mean_color = [103.64519509 / 255.0, 118.35241679 / 255.0, 124.96846096/ 255.0] 
snakes_std_color  = [50.93829403 / 255.0, 52.51745522 / 255.0, 54.89964224/ 255.0] 

# Transforms
# Resize() goes before ToTensor()
# Normalize goes after ToTensor()
transform_list = []
# Add resize transform if specified
if args.resize:
    transform_list.append(transforms.Resize((args.height, args.width)))

transform_list.append(transforms.ToTensor())
transform_list.append(transforms.Normalize(snakes_mean_color, snakes_std_color))

transform = transforms.Compose(transform_list) 

# Load and split the dataset
dataset = torchvision.datasets.ImageFolder(root=args.path, transform=transform)
print(dataset)

train_dataset, test_dataset, val_dataset = get_splits(dataset, args.train_split, args.test_split)

train_loader = data.DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)

Dataset ImageFolder
    Number of datapoints: 129939
    Root location: dataset
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.4064517454509804, 0.46412712466666667, 0.49007239592156865], std=[0.19975801580392158, 0.20595080478431374, 0.21529271466666666])
           )


In [10]:
model = None
optimizer = None
scheduler = None

args.model = 'vgg19bn'
args.weight_decay = 1e-5

if args.model == 'vgg19bn':
    model = torchvision.models.vgg19_bn(pretrained = True)
    model.classifier[6] = torch.nn.Linear(4096, n_classes)
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_step_size, gamma=args.lr_step_gamma)
elif args.model == 'vgg16':
    model = torchvision.models.vgg16(pretrained = use_pretrained) 
    model.classifier[6] = torch.nn.Linear(4096, n_classes)
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_step_size, gamma=args.lr_step_gamma)
else:
    raise Exception('Unknown model {}'.format(args.model))

criterion = F.cross_entropy

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [5]:
def train(epoch):
    '''
    Train the model for one epoch.
    '''
    # Some models use slightly different forward passes and train and test
    # time (e.g., any model with Dropout). This puts the model in train mode
    # (as opposed to eval mode) so it knows which one to use.
    
    # train loop
    for batch_idx, batch in enumerate(train_loader):
        model.train()
        # prepare data
        images, targets = Variable(batch[0]), Variable(batch[1])
        if args.cuda:
            images, targets = images.cuda(), targets.cuda()
        
        output = model(images)
        loss = F.cross_entropy(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            val_loss, val_acc = evaluate('val', n_batches=4)
            train_loss = loss.data
            examples_this_epoch = batch_idx * len(images)
            epoch_progress = 100. * batch_idx / len(train_loader)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t'
                  'Train Loss: {:.6f}\tVal Loss: {:.6f}\tVal Acc: {}'.format(
                epoch, examples_this_epoch, len(train_loader.dataset),
                epoch_progress, train_loss, val_loss, val_acc))

    scheduler.step()

    return val_acc

def evaluate(split, verbose=False, n_batches=None):
    '''
    Compute loss on val or test data.
    '''
    model.eval()
    loss = 0
    correct = 0
    n_examples = 0
    if split == 'val':
        loader = val_loader
    elif split == 'test':
        loader = test_loader
    for batch_i, batch in enumerate(loader):
        data, target = batch
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        loss += criterion(output, target, size_average=False).data
        # predict the argmax of the log-probabilities
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        n_examples += pred.size(0)
        if n_batches and (batch_i >= n_batches):
            break

    loss /= n_examples
    acc = 100. * correct / n_examples

    if verbose:
        print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            split, loss, correct, n_examples, acc))
    return loss, acc



In [11]:
acc_best = 0

# train the model one epoch at a time
for epoch in range(1, args.epochs + 1):
    val_acc = train(epoch)

    if val_acc > acc_best:
        acc_best = val_acc
        print('Saving better model ', val_acc)
        torch.save(model, args.model + '_best.pt')

evaluate('test', verbose=True)

print('Best acc ', acc_best)

e:\programs\python3.7.4\lib\site-packages\ipykernel_launcher.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


RuntimeError: CUDA out of memory. Tried to allocate 124.00 MiB (GPU 0; 8.00 GiB total capacity; 5.72 GiB already allocated; 104.25 MiB free; 91.98 MiB cached)